<a href="https://colab.research.google.com/github/DavidMercadoFaustino/deep-learning-from-scratch-python/blob/main/Projeto_(Parte_1)_Regress%C3%A3o_Linear_do_Zero_para_Pre%C3%A7os_de_Im%C3%B3veis_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
#! pip install kaggle
import zipfile
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Callable, List,Tuple, Dict

In [12]:
! mkdir ~/.kaggle
! cp /content/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [3]:
! kaggle competitions download -c house-prices-advanced-regression-techniques

In [8]:
# Mostra para Python onde estão os arquivos
caminho = os.getcwd()
# Todos arquivos NA PASTA INFORMADA
arquivos = os.listdir(caminho)

In [9]:
for arquivo in arquivos:
    if arquivo.endswith('.zip'):
        caminho_completo = os.path.join(caminho, arquivo)

In [10]:
with zipfile.ZipFile(caminho_completo, 'r') as zip_ref:
    zip_ref.extractall(caminho)

In [15]:
df = pd.read_csv('/content/train.csv')
df= df[ ['GrLivArea', 'GarageArea', 'TotalBsmtSF', '1stFlrSF', 'FullBath', 'YearBuilt', 'SalePrice']]
df.head()

,GrLivArea,GarageArea,TotalBsmtSF,1stFlrSF,FullBath,YearBuilt,SalePrice
0,1710,548,856,856,2,2003,208500
1,1262,460,1262,1262,2,1976,181500
2,1786,608,920,920,2,2001,223500
3,1717,642,756,961,1,1915,140000
4,2198,836,1145,1145,2,2000,250000


In [19]:
df.isnull().sum()

,0
GrLivArea,0
GarageArea,0
TotalBsmtSF,0
1stFlrSF,0
FullBath,0
YearBuilt,0
SalePrice,0


In [20]:
X = df.drop('SalePrice', axis=1)
y = df['SalePrice']

In [55]:
X_media = X.mean()
X_desvio = X.std()
X = (X - X_media) / X_desvio

In [56]:
y_media = y.mean()
y_desvio = y.std()
y = (y - y_media) / y_desvio

In [57]:
x = np.array(X)
y = np.array(y)

In [58]:
def mean_squared_error(predictions: np.ndarray, targets: np.ndarray) -> float:
    """Calculates the Mean Squared Error between predictions and targets."""
    # Ensure targets have the same shape as predictions for element-wise operations
    # This handles cases where targets might be (n,) and predictions (n, 1)
    targets = targets.reshape(predictions.shape)
    return np.mean(np.power(predictions - targets, 2))

def forward_linear_regression(X_batch: np.ndarray,
                              y_batch: np.ndarray,
                              weights: Dict[str, np.ndarray]) \
                              -> Tuple[float, Dict[str, np.ndarray]]:
    """
    Passagem forward para o modelo de regressão linear passo a passo.
    """
    # 1. Verificar compatibilidade das dimensões:
    # Garante que o número de observações em X_batch e y_batch são iguais
    assert X_batch.shape[0] == y_batch.shape[0], \
        f"Os tamanhos do lote de X ({X_batch.shape[0]}) e y ({y_batch.shape[0]}) devem ser iguais."

    # Garante que a multiplicação matricial (X @ W) pode ocorrer
    assert X_batch.shape[1] == weights['W'].shape[0], \
        f"O número de colunas em X_batch ({X_batch.shape[1]}) deve corresponder ao número de linhas em W ({weights['W'].shape[0]})."

    # Garante que o bias é um array 1x1 (um único valor para ser broadcastado)
    assert weights['B'].shape == (1, 1), \
        f"O bias deve ser um array 1x1. Recebido shape: {weights['B'].shape}"

    # 2. Computar as operações do passe forward:
    # N = X_batch @ W (Multiplicação Matricial)
    N = np.dot(X_batch, weights['W'])

    # P = N + B (Adição do Bias) - B is broadcasted
    P = N + weights['B']

    # Calcular a perda L (MSE)
    loss = mean_squared_error(P, y_batch)

    # 3. Salvar as informações computadas no passe forward para uso no backward pass:
    forward_info: Dict[str, np.ndarray] = {}
    forward_info['X'] = X_batch
    forward_info['N'] = N
    forward_info['P'] = P
    forward_info['y'] = y_batch

    return loss, forward_info # Retorna a perda e as informações salvas

In [59]:
np.random.seed(42)
# Pesos iniciais (aleatórios ou zeros)
weights_ex = {
    'W': np.random.randn(6, 1), # 6 features de entrada, 1 saída
    'B': np.random.randn(1, 1)  # Bias
}
current_loss, current_forward_info = forward_linear_regression(x, y, weights_ex)

In [60]:
#print(f"Dados X:\n{x.head()}")
#print(f"Alvos y:\n{y.head()}")
print(f"Pesos W:\n{weights_ex['W']}")
print(f"Bias B:\n{weights_ex['B']}")
print(f"\nPerda atual (MSE): {current_loss}")
print(f"Informações salvas (N):\n{current_forward_info['N']}")
print(f"Informações salvas (P - Previsões):\n{current_forward_info['P']}")

Pesos W:
[[ 0.49671415]
 [-0.1382643 ]
 [ 0.64768854]
 [ 1.52302986]
 [-0.23415337]
 [-0.23413696]]
Bias B:
[[1.57921282]]

Perda atual (MSE): 5.576092651596697
Informações salvas (N):
[[-1.80086914]
 [ 0.2407829 ]
 [-1.40569997]
 ...
 [ 1.21166942]
 [-0.16084364]
 [ 0.83190427]]
Informações salvas (P - Previsões):
[[-0.22165633]
 [ 1.81999572]
 [ 0.17351285]
 ...
 [ 2.79088224]
 [ 1.41836918]
 [ 2.41111709]]


In [61]:
def loss_gradients(forward_info: Dict[str, np.ndarray],
                   weights: Dict[str, np.ndarray]) -> Dict[str, np.ndarray]:
    """
    Computa dLdW e dLdB para o modelo de regressão linear passo a passo.
    """
    X_batch = forward_info['X']
    # Ensure y_batch has the same shape as predictions for element-wise operations
    # This handles cases where y_batch might be (n,) and predictions (n, 1)
    y_batch = forward_info['y'].reshape(forward_info['P'].shape)
    P = forward_info['P']


    batch_size = X_batch.shape[0]

    # 1. dLdP: Gradiente da perda L em relação às previsões P
    # L = (1/B) * sum((y - P)^2)
    # dL/dP = (2/B) * (P - y)  -- or -(2/B) * (y - P)
    # We'll use P - y for consistency with gradient descent rules
    # where we want to move in the opposite direction of the gradient.
    dLdP = (2 / batch_size) * (P - y_batch)

    # 2. dLdN: Gradiente da perda L em relação a N (where P = N + B)
    # dP/dN is a matrix of 1s (np.ones_like(N))
    # dL/dN = dL/dP * dP/dN = dL/dP * 1
    dLdN = dLdP

    # 3. dLdW: Gradiente da perda L em relação aos pesos W (where N = X @ W)
    # dN/dW is X.T
    # dL/dW = X.T @ dLdN
    dLdW = np.dot(X_batch.T, dLdN)

    # 4. dLdB: Gradiente da perda L em relação ao bias B (where P = N + B)
    # dP/dB is a matrix of 1s (np.ones_like(B) broadcasted to the shape of P)
    # dL/dB = dL/dP * dP/dB
    # However, the bias is a single value (or 1D vector) added to each observation in the batch.
    # So, we need to sum the gradients along the batch dimension (axis=0)
    # to get the total gradient for the bias.
    dLdB = np.sum(dLdP, axis=0, keepdims=True) # Keep dimensions to maintain (1,1) shape

    # Save the gradients in a dictionary for easy access
    loss_gradients: Dict[str, np.ndarray] = {}
    loss_gradients['W'] = dLdW
    loss_gradients['B'] = dLdB

    return loss_gradients

In [62]:
# # Agora calculamos os gradientes:
gradients = loss_gradients(current_forward_info, weights_ex)

print(f"\nGradientes da Perda:")
print(f"dL/dW:\n{gradients['W']}")
print(f"dL/dB:\n{gradients['B']}")


Gradientes da Perda:
dL/dW:
[[ 1.37020227]
 [ 0.64971767]
 [ 2.54478242]
 [ 3.01004129]
 [ 0.28299154]
 [-0.30223297]]
dL/dB:
[[3.15842563]]


In [65]:
# Hiperparâmetros
LEARNING_RATE = 0.001
EPOCHS = 500
BATCH_SIZE = 32

# Inicialização de pesos - Ensure weights are initialized correctly for a single output
weights_ex = {
    'W': np.random.randn(X.shape[1], 1) * 0.01, # Number of features x 1 output
    'B': np.zeros((1, 1)) # Single bias value
}

# Loop de treinamento
n_batches = int(np.ceil(len(x) / BATCH_SIZE))

for epoch in range(EPOCHS):
    epoch_loss = 0
    indices = np.random.permutation(len(x))

    for batch in range(n_batches):
        # Selecionar batch
        start = batch * BATCH_SIZE
        end = min(start + BATCH_SIZE, len(x))
        batch_idx = indices[start:end]
        x_batch = x[batch_idx]
        y_batch = y[batch_idx]

        # Forward pass
        loss, forward_info = forward_linear_regression(x_batch, y_batch, weights_ex)
        epoch_loss += loss

        # Backward pass
        grads = loss_gradients(forward_info, weights_ex)

        # Atualização de pesos
        weights_ex['W'] -= LEARNING_RATE * grads['W']
        weights_ex['B'] -= LEARNING_RATE * grads['B']

    # Log de progresso
    avg_loss = epoch_loss / n_batches
    if epoch % 10 == 0 or epoch == EPOCHS - 1: # Adjusted logging frequency
        print(f"Epoch {epoch}/{EPOCHS} - Loss: {avg_loss:.4e}")

# Avaliação no conjunto de teste
def evaluate(x_data, y_data):
    # Ensure consistent shapes for prediction
    predictions = np.dot(x_data, weights_ex['W']) + weights_ex['B']
    y_data = y_data.reshape(predictions.shape) # Reshape y_data for metrics

    mae = np.mean(np.abs(predictions - y_data))
    rmse = np.sqrt(np.mean(np.power(predictions - y_data, 2)))
    return mae, rmse

# Calcular métricas
train_mae, train_rmse = evaluate(x, y)


print("\n" + "="*50)
print("Métricas Finais:")
print(f"MAE Treino: ${train_mae:.2f}")
print(f"RMSE Treino: ${train_rmse:.2f}")
print("-"*50)
print("="*50)

# Inspeção dos pesos finais
print("\nPesos finais:")
for i, col in enumerate(X.columns):
    print(f"{col}: {weights_ex['W'][i][0]:.3f}")
print(f"Bias: {weights_ex['B'][0][0]:.3f}")

Epoch 0/500 - Loss: 8.3926e-01
Epoch 10/500 - Loss: 3.1490e-01
Epoch 20/500 - Loss: 3.0603e-01
Epoch 30/500 - Loss: 2.9705e-01
Epoch 40/500 - Loss: 2.9475e-01
Epoch 50/500 - Loss: 2.9168e-01
Epoch 60/500 - Loss: 2.8966e-01
Epoch 70/500 - Loss: 2.8830e-01
Epoch 80/500 - Loss: 2.8745e-01
Epoch 90/500 - Loss: 2.8768e-01
Epoch 100/500 - Loss: 2.8833e-01
Epoch 110/500 - Loss: 2.8643e-01
Epoch 120/500 - Loss: 2.8889e-01
Epoch 130/500 - Loss: 2.8685e-01
Epoch 140/500 - Loss: 2.8835e-01
Epoch 150/500 - Loss: 2.8747e-01
Epoch 160/500 - Loss: 2.8730e-01
Epoch 170/500 - Loss: 2.8607e-01
Epoch 180/500 - Loss: 2.8769e-01
Epoch 190/500 - Loss: 2.8701e-01
Epoch 200/500 - Loss: 2.8671e-01
Epoch 210/500 - Loss: 2.8769e-01
Epoch 220/500 - Loss: 2.8647e-01
Epoch 230/500 - Loss: 2.8631e-01
Epoch 240/500 - Loss: 2.8650e-01
Epoch 250/500 - Loss: 2.8656e-01
Epoch 260/500 - Loss: 2.8649e-01
Epoch 270/500 - Loss: 2.8600e-01
Epoch 280/500 - Loss: 2.8662e-01
Epoch 290/500 - Loss: 2.8651e-01
Epoch 300/500 - Loss: